In [54]:
from basf_measurement_parser import BASFMeasurementParser
from prompt_evaluation import *
from prompt_builder import *
from logger import ParserLogger



In [55]:
# logger = ParserLogger("Data/text_data_ipa221229_30_patents.log")

chat_prompt, prompt, examples  = PromptBuilder.build_prompt_from_dir("003")

In [56]:
prompt

{'text': "You are given a text from chemistry research patent, extract all the measurements and it's context mentioned in the text. Only Extract Measurement context, Unit and Value ranges.\nOnly extract measurements that have value and units.\nIf you find no measurements, return [].",
 'output_format': "Your output should follow be a list of json objects, good output format : [ { 'measurement ' : 'string ', 'unit ' : 'string ', 'value ' : 'string ' }, { 'measurement ' : 'string ', 'unit ' : 'string ', 'value ' : 'string ' }"}

In [57]:
measurement_parser = BASFMeasurementParser(prompt=chat_prompt, prompt_id="003", chunk_size=6000)


In [5]:
text = measurement_parser._load_txt_and_split("Data/text_data_ipa221229_5_patents.txt")

In [58]:
patents_text_results = measurement_parser.parse_txt_by_chunks(file_path="Data/text_data_ipa221229_5_patents.txt", output_format=prompt["output_format"])


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\cm_El\anaconda3\envs\langchain\Lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\cm_El\AppData\Local\Temp\ipykernel_28096\4222190929.py", line 1, in <module>
    patents_text_results = measurement_parser.parse_txt_by_chunks(file_path="Data/text_data_ipa221229_5_patents.txt", output_format=prompt["output_format"])
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\00Work\BASF\challange\basf_measurement_parser.py", line 131, in parse_txt_by_chunks
    with open(
         ^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'output/parsing_results_003_6000_false_json.txt'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\cm_El\anaconda3\envs\langch

In [59]:
patents_text_results

NameError: name 'patents_text_results' is not defined

In [4]:
file = open("output/parsing_results_54_chunks_prompt_001.txt")
prediction_test = file.read()

In [5]:
prediction_test_list = prediction_test.replace("\n", " ").split(']')
prediction_test_list[0]

'[     {         "measurement": "surface area",          "unit": "m^2/g",          "value": "between about 5 and about 350"     },      {         "measurement": "surface area",          "unit": "m/g",          "value": "between about 30 and 80"     },     {         "measurement": "coated Substrate support",          "unit": "g/in",          "value": "about 1.5 to about 7.0"     },     {         "measurement": "polypeptide concentration",          "unit": "g/ml",          "value": "1 to 9"     } '

In [16]:
import re
import ast
import string 

def build_dict_from_json_string(json_str_list):
    try:
        fixed_json = json.loads(json_str_list)
        return fixed_json
    except json.JSONDecodeError as e:
        # Remove the surrounding square brackets and newlines
        clean_string = json_str_list.replace("'", "\"")

        # Safely evaluate the string as a list of dictionaries
        dictionary_list = ast.literal_eval(clean_string)
        
        return dictionary_list

In [1]:
prediction = '''[{"measurement": "nucleic acid sequencing", "unit": "", "value": ""},
{"measurement": "rt-pcr sequencing", "unit": "", "value": ""},
{"measurement": "pacbio sequencing protocol", "unit": "", "value": ""},
{"measurement": "geographic location of infected individuals", "unit": "", "value": ""},
{"measurement": "barcodes linked to zip code or geographic identifier", "unit": "", "value": ""},
{"measurement": "prevalence of variants in infected individuals or general population", "unit": "", "value": ""},
{"measurement": "correlation of specific variants with infectivity and disease severity", "unit": "", "value": ""},
{"measurement": "data deposition in CDC database", "unit": "", "value": ""}]'''

In [8]:
prediction_test_list[30]

' As there are no measurements with units and values in the provided text, no output can be generated. "measurement": "top lance diameter", "unit": "NA", "value": "larger than sublance diameter", "measurement": "sublance diameter", "unit": "NA", "value": "NA", "measurement": "probe parameters", "unit": "NA", "value": "NA", "measurement": "sampling tool parameters", "unit": "NA", "value": "NA" I apologize, but I cannot provide an answer to your request as it seems incomplete and does not provide any text to extract measurements from. Could you please provide the appropriate text? [     {         "measurement": "diameter of top lance distal end",         "unit": null,         "value": "d1"     },     {         "measurement": "maximum diameter of top lance coupling portion",         "unit": null,         "value": "d1"     },     {         "measurement": "diameter of sublance cavity",         "unit": null,         "value": "d2"     },     {         "measurement": "diameter of external cros

In [47]:
import json
import pandas as pd
def post_process_predictions(
        predictions: list, return_df=True
    ) :
        """Parse Predictions from LLM to follow JSON format.

        We parse the string to load it as JSON, some predictions fails to be parsed as JSON,
        they need further post_processing

        Args:
            predictions (list[str]): List of LLM outputs.
            return_df (bool, optional): IF true, returned predictions will be in a DataFrame. Default True.

        Returns:
            Tuple[dict, dict]: First item is the list of json objects,
            second item is predictions failed to be parsed as Json.
        """
        predictions_json = []
        false_json = []
        for prediction in predictions:
            try:
                predictions_json.extend(build_dict_from_json_string(prediction))
            except Exception as exception:
                
                false_json.append(prediction)
        
        return predictions_json, false_json
    
def post_process_predicions_from_file(prediction_file_path):
    
    with open(prediction_file_path, encoding="utf8") as file:
        predictions = file.read()
    
    
    
    prediction_list = predictions.replace("\n", " ").split(']')
    character = ']'
    prediction_list = list(map(lambda item: item + character, prediction_list))
    
    return post_process_predictions(prediction_list)

In [48]:
predictions_json, false_jsons = post_process_predicions_from_file("parsing_results_54_chunks_prompt_002.txt")

In [50]:
pd.DataFrame(predictions_json)

,measurement,unit,value,measurement,unit,value,mea...nt,region
0,refractory metal oxide support surface area,m^2/g,5 to 350,NaN,NaN,NaN,NaN,NaN
1,refractory metal oxide support surface area,m^2/g,100 to 200,NaN,NaN,NaN,NaN,NaN
2,coated Substrate support,g/in,1.5 to 7.0,NaN,NaN,NaN,NaN,NaN
3,Alumina BET surface area,m/g,200,NaN,NaN,NaN,NaN,NaN
4,nitrogen oxide surface area,m^2/g,30 and 80,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
609,cr,%,0.5 to 1.0,NaN,NaN,NaN,NaN,NaN
610,sb,%,0.05 to 0.20,NaN,NaN,NaN,NaN,NaN
611,"cu*, cr*, and sb* composition",%,"average concentrations of cu, cr, and sb, resp...",NaN,NaN,NaN,NaN,NaN
612,yield strength,mpa,230 or more,NaN,NaN,NaN,NaN,NaN


In [ ]:
measurement
measurement

In [46]:
predictions_json

,measurement,unit,value,measurement,unit,value,mea...nt,region
0,refractory metal oxide support surface area,m^2/g,5 to 350,NaN,NaN,NaN,NaN,NaN
1,refractory metal oxide support surface area,m^2/g,100 to 200,NaN,NaN,NaN,NaN,NaN
2,coated Substrate support,g/in,1.5 to 7.0,NaN,NaN,NaN,NaN,NaN
3,Alumina BET surface area,m/g,200,NaN,NaN,NaN,NaN,NaN
4,nitrogen oxide surface area,m^2/g,30 and 80,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
609,cr,%,0.5 to 1.0,NaN,NaN,NaN,NaN,NaN
610,sb,%,0.05 to 0.20,NaN,NaN,NaN,NaN,NaN
611,"cu*, cr*, and sb* composition",%,"average concentrations of cu, cr, and sb, resp...",NaN,NaN,NaN,NaN,NaN
612,yield strength,mpa,230 or more,NaN,NaN,NaN,NaN,NaN


In [22]:
false_jsons[0]

'\n"measurement": "condensed positive plate specimens", "unit": "", "value": "96",\n"measurement": "samples in one production batch", "unit": "", "value": "768",\n"measurement": "genome coverage", "unit": "%", "value": "at least 90",\n"measurement": "sars-cov-2 genome length", "unit": "kb", "value": "30"\n[\n{ \n"measurement": "incubation step temperature", \n"unit": "c.", \n"value": "25", \n"time": "10 min" \n},\n{ \n"measurement": "incubation step temperature", \n"unit": "c.", \n"value": "50", \n"time": "50 min" \n},\n{ \n"measurement": "incubation step temperature", \n"unit": "c.", \n"value": "95", \n"time": "1 min" \n},\n{ \n"measurement": "incubation step temperature", \n"unit": "c.", \n"value": "5", \n"time": "16-24 hours" \n},\n{ \n"measurement": "clean-up incubation temperature", \n"unit": "c.", \n"value": "45", \n"time": "1 hr" \n},\n{ \n"measurement": "clean-up incubation temperature", \n"unit": "c.", \n"value": "95", \n"time": "3 min" \n},\n{ \n"measurement": "PCR incubation

In [10]:
predictions.split(']')

['[{"measurement": "refractory metal oxide support surface area", "unit": "m^2/g", "value": "5 to 350"},\n{"measurement": "refractory metal oxide support surface area", "unit": "m^2/g", "value": "100 to 200"},\n{"measurement": "coated Substrate support", "unit": "g/in", "value": "1.5 to 7.0"},\n{"measurement": "Alumina BET surface area", "unit": "m/g", "value": "200"},\n{"measurement": "nitrogen oxide surface area", "unit": "m^2/g", "value": "30 and 80"},\n{"measurement": "polypeptide concentration", "unit": "g/ml", "value": "1 to 9"}',
 '\n[ \n  { \n    "measurement": "nucleic acid sequencing", \n    "unit": "", \n    "value": "" \n  }, \n  { \n    "measurement": "rt-pcr sequencing", \n    "unit": "", \n    "value": "" \n  }, \n  { \n    "measurement": "pacbio sequencing protocol", \n    "unit": "", \n    "value": "" \n  }, \n  { \n    "measurement": "geographic location identification", \n    "unit": "", \n    "value": "" \n  }, \n  { \n    "measurement": "barcode linkage", \n    "un

In [7]:
predictions_json

[]

In [10]:
predictions_json, false_jsons = post_process_predicions(prediction_test_list)

In [11]:
import pandas as pd 

pd.DataFrame(predictions_json)

,measurement,unit,value,measurement,unit,value
0,surface area,m^2/g,between about 5 and about 350,NaN,NaN,NaN
1,surface area,m/g,between about 30 and 80,NaN,NaN,NaN
2,coated Substrate support,g/in,about 1.5 to about 7.0,NaN,NaN,NaN
3,polypeptide concentration,g/ml,1 to 9,NaN,NaN,NaN
4,nucleic acid sequencing,,,NaN,NaN,NaN
...,...,...,...,...,...,...
498,yield strength,mpa,230 or more,NaN,NaN,NaN
499,tensile strength,mpa,380 or more,NaN,NaN,NaN
500,heating temperature,c,1.0 to 1.20,NaN,NaN,NaN
501,hot rolling temperature,c,80 or more,NaN,NaN,NaN


In [12]:
predictions_json = []
false_jsons =[]
for i, prediction in enumerate(prediction_test_list):
    try:
        predictions_json.extend(build_dict_from_json_string(prediction+']'))
    except Exception as e:
        print(e)
        false_jsons.append(prediction)

invalid syntax (<unknown>, line 1)
invalid syntax (<unknown>, line 1)
invalid syntax (<unknown>, line 1)
invalid syntax (<unknown>, line 1)
'{' was never closed (<unknown>, line 1)
unmatched ']' (<unknown>, line 1)


In [13]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# The schema I want out
response_schemas = [
    ResponseSchema(name="measurements", description="Measurements details"),
    ResponseSchema(name="unit", description="Unit used in the measurments"),
    ResponseSchema(name="value", description="The value of the measurements")
]

# The parser that will look for the LLM output in my schema and return it back to me
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [23]:
from dotenv import load_dotenv
load_dotenv("env.env")


True

In [20]:
# Build vectorstore and keep the metadata
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=all_splits,
                                    embedding=OpenAIEmbeddings(deployment_name=""))

NameError: name 'all_splits' is not defined